In [1]:
# importing libraries
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"]

In [5]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
# VectorStoreIndex helps to convert all the text into vector and index them
documents = SimpleDirectoryReader("data").load_data()

In [6]:
documents

[Document(id_='edd0db0d-260e-482b-bd9d-99514356d110', embedding=None, metadata={'page_label': '1', 'file_name': 'apjspeech.pdf', 'file_path': 'data\\apjspeech.pdf', 'file_type': 'application/pdf', 'file_size': 37862, 'creation_date': '2024-02-17', 'last_modified_date': '2024-02-14', 'last_accessed_date': '2024-02-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \

In [7]:
# Now converting into index
index = VectorStoreIndex.from_documents(documents, show_progress=True)


Parsing nodes:   0%|          | 0/65 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/65 [00:00<?, ?it/s]

In [8]:
index

In [9]:
# now can we query the database
query_engine = index.as_query_engine()

In [17]:
response = query_engine.query("Who is Abdul Kalam, and tell me his taughts")
print(response)

Abdul Kalam is a prominent figure who completed five years in Rashtrapati Bhavan. He shared important messages during his tenure, such as accelerating development, empowering villages, mobilizing rural core competence, and fostering a youth movement for a Developed India by 2020. He emphasized the importance of connecting with the people of India and instilling self-confidence in achieving national development goals.


In [22]:
# Method 1
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response, show_source=True)

Final Response: Abdul Kalam is a prominent figure who completed five
years in Rashtrapati Bhavan. He shared important messages during his
tenure, such as accelerating development, empowering villages,
mobilizing rural core competence, and fostering a youth movement for a
Developed India by 2020. He emphasized the importance of connecting
with the people of India and instilling self-confidence in achieving
national development goals.
______________________________________________________________________
Source Node 1/2
Node ID: 929e9aef-d806-4050-b071-a0ffb12efc7c
Similarity: 0.8293802831149718
Text: A P J Abdul Kalam Departing speech      Friends, I am delighted
to address you all, in the country and those livi ng abroad, after
working with you and completing five beautiful and eventful years in
Rashtrapati  Bhavan. Today, it is indeed a thanks giving occasion. I
would like to narr ate, how I  enjoyed every minute of my tenure
enriched by ...
___________________________________________

In [36]:
# Method 2
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# giving suggestions to respond with 4 results
retreiver = VectorIndexRetriever(index=index, similarity_top_k=4)
postprocessor = SimilarityPostprocessor(similarity_threshold=0.5, similarity_cutoff=0.80)
query_engine = RetrieverQueryEngine(retriever=retreiver, node_postprocessors=[postprocessor])

In [37]:
response = query_engine.query("Who is Abdul Kalam, and tell me his taughts")
print(response)

Abdul Kalam is a prominent figure who completed five years at Rashtrapati Bhavan. He expressed gratitude towards the diverse associations he had during his tenure, including individuals from various fields like politics, science, technology, arts, and more. Kalam emphasized messages such as accelerating development, empowering villages, mobilizing rural competence, and fostering a youth movement for a Developed India by 2020. He shared a memorable interaction with a young girl questioning why India couldn't become a developed nation before 2020. Kalam's mission was to unite the hearts and minds of the people of India and instill the belief that the nation can achieve greatness.


In [38]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response, show_source=True)

Final Response: Abdul Kalam is a prominent figure who completed five
years at Rashtrapati Bhavan. He expressed gratitude towards the
diverse associations he had during his tenure, including individuals
from various fields like politics, science, technology, arts, and
more. Kalam emphasized messages such as accelerating development,
empowering villages, mobilizing rural competence, and fostering a
youth movement for a Developed India by 2020. He shared a memorable
interaction with a young girl questioning why India couldn't become a
developed nation before 2020. Kalam's mission was to unite the hearts
and minds of the people of India and instill the belief that the
nation can achieve greatness.
______________________________________________________________________
Source Node 1/2
Node ID: 929e9aef-d806-4050-b071-a0ffb12efc7c
Similarity: 0.8293802831149718
Text: A P J Abdul Kalam Departing speech      Friends, I am delighted
to address you all, in the country and those livi ng abroad, af

In [39]:
# Here we are storing the index into my local storage, then using it to query the vector
# because of the memory utilization, we are doing like this
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What are transformers?")
print(response)

Transformers are devices that are used to transfer electrical energy between two or more circuits through electromagnetic induction. They typically consist of two coils of wire, known as the primary and secondary coils, which are wrapped around a core made of ferromagnetic material. Transformers are commonly used in electrical power distribution systems to step up or step down voltage levels for efficient transmission and distribution of electricity.
